In [1]:
# (a, b, c, d)
effects = [
    (1, 0, 0, 0), # main effect 0:4
    (0, 1, 0, 0), 
    (0, 0, 1, 0), 
    (0, 0, 0, 1),
    (1, 1, 0, 0), # two-factor interaction 4:16
    (1, 2, 0, 0),
    (1, 0, 1, 0),
    (1, 0, 2, 0),
    (1, 0, 0, 1),
    (1, 0, 0, 2),
    (0, 1, 1, 0),
    (0, 1, 2, 0),
    (0, 1, 0, 1), 
    (0, 1, 0, 2),
    (0, 0, 1, 1),
    (0, 0, 1, 2), 
    (1, 1, 1, 0), # three-factor interaction 16: 32
    (1, 1, 2 ,0),
    (1, 2, 1, 0),
    (1, 2, 2, 0),
    (1, 1, 0, 1),
    (1, 1, 0, 2),
    (1, 2, 0, 1),
    (1, 2, 0, 2),
    (1, 0, 1, 1),
    (1, 0, 1, 2),
    (1, 0, 2, 1),
    (1, 0, 2, 2),
    (0, 1, 1, 1),
    (0, 1, 1, 2),
    (0, 1, 2, 1),
    (0, 1, 2, 2),
    (1, 1, 1, 1), # four-factor interaction 32:40
    (1, 1, 1, 2),
    (1, 1, 2, 1),
    (1, 1, 2, 2),
    (1, 2, 1, 1),
    (1, 2, 1, 2),
    (1, 2, 2, 1),
    (1, 2, 2, 2)  
]

40

In [2]:
import numpy as np
effects = [np.array(effect) for effect in effects]

main_effect = effects[0:4]
two_fi = effects[4:16]
three_fi = effects[16:32]
four_fi = effects[32:40]

In [3]:
import numpy as np

def offseting_effect(effect: np.ndarray) -> np.ndarray:
    # Find the index of the first non-zero element
    non_zero_indices = np.nonzero(effect)[0]
    if len(non_zero_indices) > 0:
        first_non_zero_index = non_zero_indices[0]
        first_non_zero_value = effect[first_non_zero_index]
        
        # If the first non-zero digit is not 1, multiply the array by 2 and mod 3
        if first_non_zero_value != 1:
            return (effect * 2) % 3
    
    # Return the original array if the first non-zero digit is 1 or the array is all zeros
    return effect


def generate_design_C(design_A: list[np.ndarray], design_B: list[np.ndarray]) -> list[np.ndarray]:
    if len(design_A) != len(design_B):
        raise ValueError("design_A and design_B must have the same length.")

    design_C = []
    for element_A, element_B in zip(design_A, design_B):
        # Perform the element-wise operation
        element_C = (element_A + element_B) % 3
        # Apply the offseting_effect_np function to C
        D_transformed = element_C
        design_C.append(D_transformed)
        
    return design_C

def generate_design_D(design_A: list[np.ndarray], design_B: list[np.ndarray]) -> list[np.ndarray]:
    if len(design_A) != len(design_B):
        raise ValueError("design_A and design_B must have the same length.")

    design_D = []
    for element_A, element_B in zip(design_A, design_B):
        # Perform the element-wise operation
        element_D = (element_A + element_B * 2) % 3
        # Apply the offseting_effect_np function to D
        D_transformed = offseting_effect(element_D)
        design_D.append(D_transformed)
        
    return design_D

In [5]:
from itertools import combinations
from tqdm.notebook import tqdm


def filter_effects(effects, design_A):
    """Filter out arrays in design_A from effects."""
    remaining_effects = []
    for effect in effects:
        if not any(np.array_equal(effect, a) for a in design_A):
            remaining_effects.append(effect)
    return remaining_effects

def has_repeated_array(*lists_of_arrays):
    """Check if there are any repeated arrays across multiple lists of arrays."""
    checked_arrays = []
    for list_of_arrays in lists_of_arrays:
        for arr in list_of_arrays:
            # Check if arr is equal to any array we've already checked
            if any(np.array_equal(arr, checked_arr) for checked_arr in checked_arrays):
                return True
            checked_arrays.append(arr)
    return False


for selected_four_fi in combinations(four_fi, 6):
    
    pbar = tqdm(total=30045015)
    
    design_A = main_effect + list(selected_four_fi)
    remaining_effects = filter_effects(effects, design_A)
    
    # Then select 10 designs from the remaining for group B
    for design_B in combinations(remaining_effects, 10):
        
        design_B = list(design_B)        
        design_C = generate_design_C(design_A, design_B)
        design_D = generate_design_D(design_A, design_B)

        if not has_repeated_array(design_A, design_B, design_C, design_D):
            print("good!")
        pbar.update(1)
    


  0%|          | 0/30045015 [00:00<?, ?it/s]